In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import seq2seq

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [8]:
importlib.reload(seq2seq)    
net = seq2seq.Transformer(char2i, i2char)
net.to(device)
net.fit(source_1, target_1, save_path = "checkpoints/seq2seq_transformer.pt")

Model: Transformer
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Embedding dimension: 8
Feedforward dimension: 32
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 6,759

Epoch | Train                 | Training time
      | Loss     | Error Rate |
---------------------------------------------
    1 |   2.8016 |     61.004 |        112.50
    2 |   2.1414 |     55.147 |        226.57
    3 |   1.9693 |     52.035 |        349.77
    4 |   1.8914 |     50.299 |        459.14
    5 |   1.8468 |     49.378 |        571.56


,epoch,train_loss,train_error_rate,training_time,learning_rate,weight_decay,model,embedding_dimension,feedforward_dimension,encoder_layers,decoder_layers,attention_heads,activation,dropout,parameters
0,1,2.801586,61.003737,112.498123,0.0001,0,Transformer,8,32,2,2,2,relu,0.0,6759
1,2,2.141373,55.146563,226.566216,0.0001,0,Transformer,8,32,2,2,2,relu,0.0,6759
2,3,1.969328,52.034820,349.772037,0.0001,0,Transformer,8,32,2,2,2,relu,0.0,6759
3,4,1.891375,50.299033,459.136350,0.0001,0,Transformer,8,32,2,2,2,relu,0.0,6759
4,5,1.846799,49.378072,571.563894,0.0001,0,Transformer,8,32,2,2,2,relu,0.0,6759


In [9]:
importlib.reload(seq2seq)    
net = seq2seq.Transformer(char2i, i2char)
net.load_state_dict(torch.load("checkpoints/seq2seq_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(source_1[:5])

[net.tensor2text(t) for t in indexes]

Model: Transformer
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Embedding dimension: 8
Feedforward dimension: 32
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 6,759



[['<START>s the the the  th th',
  '<START>s the the the  tt th',
  '<START>s the the the  th he',
  '<START>s the the the  tt he',
  '<START>s the the the  th at'],
 ['<START> the te the the t th',
  '<START> the te the the t at',
  '<START> the te the the t\n t',
  '<START> the te the then het',
  '<START> the te the the chet'],
 ['<START> the te the the t th',
  '<START> the te the the t at',
  '<START> the te the the t\n t',
  '<START> the te the the t\n  ',
  '<START> the te the the  het'],
 ['<START>the the the the t th',
  '<START>the the the the athe',
  '<START>the the the the  het',
  '<START>the the the the atho',
  '<START>the the the the  hes'],
 ['<START>the the the the t th',
  '<START>the the the the  het',
  '<START>the the the the t\n t',
  '<START>the the the the t\n  ',
  '<START>the the the the  hes']]

In [10]:
importlib.reload(seq2seq)    
net = seq2seq.Transformer(char2i, i2char)
net.load_state_dict(torch.load("checkpoints/seq2seq_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.sample(source_1[:5])

net.tensor2text(indexes)

Model: Transformer
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
Embedding dimension: 8
Feedforward dimension: 32
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 6,759



['<START>\n\nTrcatonicarsotuck.',
 "<START>gafir of s'kos fv ov",
 '<START>ged P-f. Ehifiecmede',
 '<START>amvifi Whafbe o. t;o',
 '<START>an te\nlosevonehwos Q']